In [ ]:
#imports

from google.colab import files
import pandas as pd
import numpy as np

# ACS 5-Year 2025 Means of Transport Loading/Cleaning



In [ ]:
# Load dataset
acs = pd.read_csv('/content/ACS_5Year_2023_Means_of_Transport.csv')

# Get a list of all column names
all_columns = acs.columns.tolist()

# Filter columns to keep only "Estimate" columns
estimate_columns = [col for col in all_columns if 'Estimate' in col and 'Margin of Error' not in col]

# Keep the "Label (Grouping)" column and the selected "Estimate" columns
acs = acs[['Label (Grouping)'] + estimate_columns]

In [ ]:
# keep only bike data and total commuter data
acs = acs.iloc[[0, 17]]
#print(acs.head(1))


In [ ]:
# Convert all values to numeric from strings, ignoring the first column
for i in range(acs.shape[0]):
    for j in range(1, acs.shape[1]):  # Start from column 1
        try:
            # Replace commas in string numbers before conversion
            value = acs.iloc[i, j]
            if isinstance(value, str):
                value = value.replace(',', '')
            acs.iloc[i, j] = pd.to_numeric(value, errors='coerce')
        except ValueError:
            # Handle non-numeric strings (e.g., keep them as strings or assign NaN)
            pass  # Or acs.iloc[i, j] = np.nan


In [ ]:
# Define the new row data with the string label
new_row = ['% Bike Commuter']

# Extend the new row with zeros to match the number of numeric columns
new_row.extend([0] * (len(acs.columns) - 1))

# Create a new DataFrame with the new row
new_row_df = pd.DataFrame([new_row], columns=acs.columns)

# Append the new row DataFrame to the original DataFrame
acs = pd.concat([acs, new_row_df], ignore_index=True)


In [ ]:
# Change the data type of all columns except the first one to float64
for col in acs.columns[1:]:
    acs[col] = acs[col].astype(float)  # Convert to float64

for i in range(1, len(acs.columns)):
    try:
        # Get numerator and denominator
        numerator = pd.to_numeric(acs.iloc[1, i])
        denominator = pd.to_numeric(acs.iloc[0, i])

        # Check for zero denominator or NaN values
        if denominator == 0 or np.isnan(numerator) or np.isnan(denominator):
            acs.iloc[2, i] = 0 if denominator == 0 else np.nan  # Assign 0 if total is 0, NaN otherwise
        else:
            # Cast the result to float64 before assigning
            acs.iloc[2, i] = ((numerator / denominator) * 100).astype(float)  # This line should no longer cause a warning

    except (ValueError, TypeError):
        acs.iloc[2, i] = np.nan  # Assign NaN for other errors

In [ ]:
import re

def extract_tract_and_convert_to_geoid(value):
    """
    Extracts the tract number from a string and converts it to a full GEOID,
    replacing the entire cell value.

    Args:
        value: A string in the format "Census Tract 101.01; San Francisco County; California!!Estimate"
               or "Census Tract 106; San Francisco County; California!!Estimate"

    Returns:
        The full GEOID (e.g., 6075010101) or the original value if no tract is found.
    """
    # Ensure value is a string before applying regex
    if isinstance(value, bytes):
        value = value.decode('utf-8', errors='ignore')
    elif not isinstance(value, str):  # Handle non-string, non-bytes values
        return value

    # Updated regex to handle tracts with or without decimals
    match = re.search(r"Census Tract (\d+(?:\.\d+)?)", value)
    if match:
        tract_number = match.group(1).replace(".", "")  # Remove the decimal if present
        full_geoid = "06075" + tract_number.zfill(6)  # Add state and county FIPS, pad with zeros
        return full_geoid  # Return only the GEOID if found
    return value  # Return original value if no match

# Apply the function directly to the column names (header)
acs.columns = acs.columns.map(extract_tract_and_convert_to_geoid)

# Print the DataFrame to verify changes
print(acs.head(0))  # Print only the header row

Empty DataFrame
Columns: [Label (Grouping), 06075010101, 06075010102, 06075010201, 06075010202, 06075000103, 06075010401, 06075010402, 06075000105, 06075000106, 06075010701, 06075010702, 06075000108, 06075010901, 06075010902, 06075011001, 06075011002, 06075011101, 06075011102, 06075000112, 06075000113, 06075000117, 06075000118, 06075011901, 06075011902, 06075012001, 06075012002, 06075000121, 06075012202, 06075012203, 06075012204, 06075012301, 06075012302, 06075012403, 06075012404, 06075012405, 06075012406, 06075012502, 06075012503, 06075012504, 06075012601, 06075012602, 06075000127, 06075012801, 06075012802, 06075012901, 06075012902, 06075013001, 06075013002, 06075013101, 06075013102, 06075000132, 06075000133, 06075013401, 06075013402, 06075000135, 06075000151, 06075015201, 06075015202, 06075000153, 06075015401, 06075015402, 06075000155, 06075000156, 06075015701, 06075015702, 06075015801, 06075015802, 06075000159, 06075000160, 06075016101, 06075016102, 06075000162, 06075000163, 0607500

In [ ]:
acs.to_csv('ACS_cleaned.csv', index=False)
files.download('ACS_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Crashes Loading/Cleaning

In [ ]:
# Load dataset
crashes = pd.read_csv('/content/Crashes.csv')
# Get a list of all column names
all_columns = crashes.columns.tolist()
print(all_columns)
#print(crashes.head())


['CASE_ID', 'ACCIDENT_YEAR', 'PROC_DATE', 'JURIS', 'COLLISION_DATE', 'COLLISION_TIME', 'OFFICER_ID', 'REPORTING_DISTRICT', 'DAY_OF_WEEK', 'CHP_SHIFT', 'POPULATION', 'CNTY_CITY_LOC', 'SPECIAL_COND', 'BEAT_TYPE', 'CHP_BEAT_TYPE', 'CITY_DIVISION_LAPD', 'CHP_BEAT_CLASS', 'BEAT_NUMBER', 'PRIMARY_RD', 'SECONDARY_RD', 'DISTANCE', 'DIRECTION', 'INTERSECTION', 'WEATHER_1', 'WEATHER_2', 'STATE_HWY_IND', 'CALTRANS_COUNTY', 'CALTRANS_DISTRICT', 'STATE_ROUTE', 'ROUTE_SUFFIX', 'POSTMILE_PREFIX', 'POSTMILE', 'LOCATION_TYPE', 'RAMP_INTERSECTION', 'SIDE_OF_HWY', 'TOW_AWAY', 'COLLISION_SEVERITY', 'NUMBER_KILLED', 'NUMBER_INJURED', 'PARTY_COUNT', 'PRIMARY_COLL_FACTOR', 'PCF_CODE_OF_VIOL', 'PCF_VIOL_CATEGORY', 'PCF_VIOLATION', 'PCF_VIOL_SUBSECTION', 'HIT_AND_RUN', 'TYPE_OF_COLLISION', 'MVIW', 'PED_ACTION', 'ROAD_SURFACE', 'ROAD_COND_1', 'ROAD_COND_2', 'LIGHTING', 'CONTROL_DEVICE', 'CHP_ROAD_TYPE', 'PEDESTRIAN_ACCIDENT', 'BICYCLE_ACCIDENT', 'MOTORCYCLE_ACCIDENT', 'TRUCK_ACCIDENT', 'NOT_PRIVATE_PROPERTY', '

In [ ]:
print(crashes[['WEATHER_1', 'WEATHER_2']].head(10))  # Prints the first 10 rows


  WEATHER_1 WEATHER_2
0         A         -
1         A         -
2         A         -
3         A         -
4         B         -
5         A         -
6         A         -
7         A         -
8         A         -
9         A         -


In [ ]:

#keep 'ALCOHOL_INVOLVED' 'COLLISION_TIME' 'DAY_OF_WEEK' 'LOCATION_TYPE'  'COLLISION_SEVERITY', 'NUMBER_KILLED', 'NUMBER_INJURED', 'PARTY_COUNT', 'LATITUDE', 'LONGITUDE''POINT_X', 'POINT_Y'
columns_to_keep = ['ALCOHOL_INVOLVED', 'COLLISION_TIME', 'PRIMARY_RD', 'SECONDARY_RD', 'DAY_OF_WEEK', 'LOCATION_TYPE', 'COLLISION_SEVERITY', 'NUMBER_KILLED', 'NUMBER_INJURED', 'PARTY_COUNT', 'LATITUDE', 'LONGITUDE', 'POINT_X', 'POINT_Y']
crashes = crashes[columns_to_keep]

all_columns = crashes.columns.tolist()
#print(all_columns)
#remove rows where 'ALCOHOL_INVOLVED' and delete col
filtered_crashes = crashes[crashes['ALCOHOL_INVOLVED'].isna()]
filtered_crashes = filtered_crashes.drop('ALCOHOL_INVOLVED', axis=1)
#print(crashes.head())


In [ ]:

#print(filtered_crashes[['LATITUDE', 'LONGITUDE', 'POINT_X', 'POINT_Y']])
filtered_crashes.loc[(filtered_crashes['POINT_X'].isna()) & (filtered_crashes['LONGITUDE'].notna()), 'POINT_X'] = filtered_crashes.loc[(filtered_crashes['POINT_X'].isna()) & (filtered_crashes['LONGITUDE'].notna()), 'LONGITUDE']
filtered_crashes.loc[(filtered_crashes['POINT_Y'].isna()) & (filtered_crashes['LATITUDE'].notna()), 'POINT_Y'] = filtered_crashes.loc[(filtered_crashes['POINT_Y'].isna()) & (filtered_crashes['LATITUDE'].notna()), 'LATITUDE']

nan_pointx_or_pointy = filtered_crashes[(filtered_crashes['POINT_X'].isna()) | (filtered_crashes['POINT_Y'].isna())]
# all rows have both NaN so remove extra columns and rename
#print(nan_pointx_or_pointy)
# Remove LATITUDE and LONGITUDE columns
filtered_crashes = filtered_crashes.drop(['LATITUDE', 'LONGITUDE'], axis=1)
# Rename POINT_X to LATITUDE and POINT_Y to LONGITUDE
filtered_crashes = filtered_crashes.rename(columns={'POINT_Y': 'LATITUDE', 'POINT_X': 'LONGITUDE'})


# Individually adjusted rows missing coordinates
################################################################################
# Drop row 12 cross streets make no sense
filtered_crashes.drop(index=12, inplace=True)
# Adjust row 64 according to google maps
filtered_crashes.loc[64, 'LATITUDE'] = 37.738925
filtered_crashes.loc[64, 'LONGITUDE'] = -122.480794
# Adjust row 73 according to google maps
filtered_crashes.loc[73, 'LATITUDE'] = 37.73543018151562
filtered_crashes.loc[73, 'LONGITUDE'] = -122.50539991980963
# Adjust row 116 according to google maps
filtered_crashes.loc[116, 'LATITUDE'] = 37.78328187388983
filtered_crashes.loc[116, 'LONGITUDE'] = -122.40256946119997
# Adjust row 168 according to google maps
filtered_crashes.loc[168, 'LATITUDE'] = 37.7686225665498
filtered_crashes.loc[168, 'LONGITUDE'] = -122.50026806730958
# Adjust row 177 according to google maps- Assumption that Hobart st
# was actually Howard as this is the closest name of a cross street
filtered_crashes.loc[177, 'LATITUDE'] = 37.79244097342779
filtered_crashes.loc[177, 'LONGITUDE'] = -122.39124525261985
# Adjust row 214 according to google maps
filtered_crashes.loc[214, 'LATITUDE'] = 37.74519409803586
filtered_crashes.loc[214, 'LONGITUDE'] = -122.45132946120158
# Adjust row 232 according to google maps
filtered_crashes.loc[232, 'LATITUDE'] = 37.76830095049659
filtered_crashes.loc[232, 'LONGITUDE'] = -122.50051460352887
# Adjust row 320 according to google maps
filtered_crashes.loc[320, 'LATITUDE'] = 37.73243837321172
filtered_crashes.loc[320, 'LONGITUDE'] = -122.37537330329208


In [ ]:

# Adding Census block for each datapoint
filtered_crashes['CENSUS_TRACT'] = np.nan
filtered_crashes['CENSUS_TRACT'] = filtered_crashes['CENSUS_TRACT'].astype(float)


In [ ]:
# Check the data type of the 'LATITUDE' column
latitude_dtype = filtered_crashes['LATITUDE'].dtype

# Check the data type of the 'LONGITUDE' column
longitude_dtype = filtered_crashes['LONGITUDE'].dtype

print("Latitude Data Type:", latitude_dtype)
print("Longitude Data Type:", longitude_dtype)

Latitude Data Type: float64
Longitude Data Type: float64


In [ ]:
!pip install --upgrade requests

In [ ]:
import requests


def get_census_tract(lat, lon):

    """
    Retrieves the Census Tract (GEOID) for given latitude and longitude coordinates using the Census geocoder API.
    Prints the full API response for debugging.
    """

    api_url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={lon}&y={lat}&benchmark=Public_AR_Current&vintage=Current_Current&format=json"
    response = requests.get(api_url)

  #  print(f"API Response for ({lat}, {lon}):")  # Print for debugging
  #  print(response.json())

    if response.status_code == 200:
        result = response.json()

        # Extract the GEOID from Census Tracts, if available
        if result.get('result', {}).get('geographies', {}).get('Census Tracts'):
            return result['result']['geographies']['Census Tracts'][0]['GEOID']
        else:
            print(f"No Census Tracts data found for: {lat}, {lon}")
            return None  # Return None for missing data

    else:
        print(f"Request failed for: {lat}, {lon} with status code: {response.status_code}")
        return None  # Return None for failed requests


In [ ]:
# Iterate through the DataFrame and assign Census Tracts
for index, row in filtered_crashes.iterrows():
    lat = row['LATITUDE']
    lon = row['LONGITUDE']
    census_tract = get_census_tract(lat, lon)
    filtered_crashes.loc[index, 'CENSUS_TRACT'] = census_tract

print(filtered_crashes.head())

<ipython-input-19-dd70501cca55>:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '06075060100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  filtered_crashes.loc[index, 'CENSUS_TRACT'] = census_tract


   COLLISION_TIME                      PRIMARY_RD  \
0            1720  US-101 N/B (GGB EAST SIDEWALK)   
1            1646                      US-101 S/B   
2             851          MISSION ST. AT 6TH ST.   
3            1449                     BAYSHORE BL   
4             800                       SUNSET BL   

                     SECONDARY_RD  DAY_OF_WEEK LOCATION_TYPE  \
0  GOLDEN GATE BRIDGE SOUTH TOWER            2           NaN   
1               ALEXANDER AVE U/C            4           NaN   
2                         7TH ST.            4           NaN   
3                    SUNNYDALE AV            6           NaN   
4                      VICENTE ST            1           NaN   

   COLLISION_SEVERITY  NUMBER_KILLED  NUMBER_INJURED  PARTY_COUNT   LONGITUDE  \
0                   2              0               1            1 -122.477943   
1                   2              0               1            1 -122.479530   
2                   3              0               1 

In [ ]:
print(filtered_crashes.loc[0, 'CENSUS_TRACT'])

06075060100


In [ ]:
filtered_crashes.to_csv('Crashes_cleaned.csv', index=False)
files.download('Crashes_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


# Bike Features Loading/Cleaning


In [ ]:
# Load dataset
bikeways = pd.read_csv('/content/SFMTA_Bikeway_2025.csv')
# Get a list of all column names
#all_columns = bikeways.columns.tolist()
#print(all_columns)

# Get the value of 'shape' column in the first row
shape_value = bikeways.iloc[0]['shape']

# Print the value
print(shape_value)

POINT (-122.404748493 37.774281041)


In [ ]:
import re

# Define a function to extract coordinates
def extract_coordinates(shape_string):
    match = re.search(r"POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)", shape_string)
    if match:
        longitude = float(match.group(1))
        latitude = float(match.group(2))
        return longitude, latitude
    else:
        return None, None

# Apply the function to create new columns
bikeways[['LONGITUDE', 'LATITUDE']] = bikeways['shape'].apply(lambda x: pd.Series(extract_coordinates(x)))

# Print the updated DataFrame
print(bikeways.head())

   OBJECTID         CNN CORNER          STREET1         STREET2  \
0      2246  23875000.0      E           7TH ST       BRYANT ST   
1      4188  24570000.0     SE  THE EMBARCADERO        BROADWAY   
2        29  25474000.0    NaN  THE EMBARCADERO  NORTH POINT ST   
3      5475  34141000.0     NE       MACALLA RD  YERBA BUENA RD   
4      5133  20725000.0     NW        EVANS AVE     NAPOLEON ST   

               DESCRIPT  COUNT  INSTALL_MO  INSTALL_YR  UPDATE_MO  UPDATE_YR  \
0           BIKE SIGNAL    2.0         4.0      2020.0        NaN        NaN   
1        TWO-STAGE LEFT    1.0         3.0      2022.0        NaN        NaN   
2  INTERSECTION SHARROW    2.0         6.0      2017.0        NaN        NaN   
3              BIKE BOX    1.0         1.0      2023.0        NaN        NaN   
4           MIXING ZONE    1.0        10.0      2022.0        NaN        NaN   

                                 shape  SF Find Neighborhoods 2  \
0  POINT (-122.404748493 37.774281041)           

In [ ]:
# Add a new column for Census Tract, initializing with NaN
bikeways['CENSUS_TRACT'] = np.nan

# Iterate through the DataFrame and assign Census Tracts
for index, row in bikeways.iterrows():
    lat = row['LATITUDE']
    lon = row['LONGITUDE']
    census_tract = get_census_tract(lat, lon)
    bikeways.loc[index, 'CENSUS_TRACT'] = census_tract

print(bikeways.head())

In [ ]:
bikeways.to_csv('Bikeways_cleaned.csv', index=False)
files.download('Bikeways_cleaned.csv')

# ACS 5-year 2023 Median Income Loading/Cleaning


In [ ]:
# Load dataset
Income = pd.read_csv('/content/ACSST5Y2023.S0701.income.csv')
# Get a list of all column names
print(Income.head())


                 GEO_ID                                               NAME  \
0             Geography                               Geographic Area Name   
1  1400000US06075010101  Census Tract 101.01; San Francisco County; Cal...   
2  1400000US06075010102  Census Tract 101.02; San Francisco County; Cal...   
3  1400000US06075010201  Census Tract 102.01; San Francisco County; Cal...   
4  1400000US06075010202  Census Tract 102.02; San Francisco County; Cal...   

                                S0701_C01_001E  \
0  Estimate!!Total!!Population 1 year and over   
1                                         2004   
2                                         1781   
3                                         2608   
4                                         1749   

                                      S0701_C01_001M  \
0  Margin of Error!!Total!!Population 1 year and ...   
1                                                297   
2                                                340   
3     

In [ ]:
# Get the first row of the DataFrame
first_row = Income.iloc[0]

# Use the Income DataFrame's columns attribute directly
median_columns = [col for col in Income.columns if "Estimate!!Total!!INDIVIDUAL INCOME IN THE PAST" in str(Income[col].iloc[0]) and "Median income (dollars)" in str(Income[col].iloc[0])]

# Print the list of columns
print(median_columns)

# Select the desired columns and the first 2 rows
subset_df = Income[median_columns]
print(subset_df)

['S0701_C01_048E']
                                        S0701_C01_048E
0    Estimate!!Total!!INDIVIDUAL INCOME IN THE PAST...
1                                                73156
2                                                31184
3                                               101007
4                                                94715
..                                                 ...
240                                              13078
241                                              68456
242                                              53900
243                                                  -
244                                                  -

[245 rows x 1 columns]


In [ ]:
# Use the Income DataFrame's columns attribute directly
median_columns = [col for col in Income.columns if "Estimate!!Total!!INDIVIDUAL INCOME IN THE PAST" in str(Income[col].iloc[0]) and "Median income (dollars)" in str(Income[col].iloc[0])]

# Get the name of the first column
first_column_name = Income.columns[0]

# Create a new DataFrame with the first column and columns from median_columns (all rows)
new_df = Income[[first_column_name] + median_columns]


# Remove the first row
new_df = new_df.iloc[1:]  # Select rows from index 1 onwards

# Rename the first and second columns
new_df = new_df.rename(columns={new_df.columns[0]: "Census Tract", new_df.columns[1]: "Median Income"})

# Print the new DataFrame
print(new_df)

             Census Tract Median Income
1    1400000US06075010101         73156
2    1400000US06075010102         31184
3    1400000US06075010201        101007
4    1400000US06075010202         94715
5    1400000US06075010300         99279
..                    ...           ...
240  1400000US06075980501         13078
241  1400000US06075980600         68456
242  1400000US06075980900         53900
243  1400000US06075990100             -
244  1400000US06075990200             -

[244 rows x 2 columns]


In [ ]:
import pandas as pd

def format_census_tract(tract_id):
  try:
    # Extract the part after 'US'
    extracted_id = tract_id.split('US')[1]

    # Pad with zeros if necessary
    formatted_id = extracted_id.zfill(11)  # Pad to 11 digits with leading zeros

    return formatted_id
  except IndexError:
    # Handle cases where 'US' is not found
    return tract_id

# Apply the function to the 'Census Tract' column
new_df['Census Tract'] = new_df['Census Tract'].apply(format_census_tract)

nan_rows = new_df[new_df.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [Census Tract, Median Income]
Index: []


In [ ]:
new_df.to_csv('Income_cleaned.csv', index=False)
files.download('Income_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>